# uTensor End-to-End Workflow - Preview

## Imports

In [1]:
from utensor_cgen.api.export import tflm_keras_export

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

AUTOTUNE=tf.data.experimental.AUTOTUNE

import pathlib

dataset_url = "file:///C:/Users/jerickson/Documents/CS688/Dataset.zip"
data_dir = tf.keras.utils.get_file(origin=dataset_url,fname='Dataset_file',extract=True)

data_dir = pathlib.Path(data_dir)

data_dir_string = str(data_dir).replace('_file','')

data_dir_string_path = pathlib.Path(data_dir_string)

CLASS_NAMES = np.array([item.name for item in data_dir_string_path.glob('*') if item.name != "LICENSE.txt"])

print(CLASS_NAMES)

#image_count_array = np.array([item.name for item in data_dir_string_path.glob('*/*.jpg') if item.name != "LICENSE.txt"])
#print(image_count_array.size)

image_count = len(list(data_dir_string_path.glob('*/*.jpg')))

print(image_count)

BATCH_SIZE = 32
IMG_HEIGHT = 28
IMG_WIDTH = 28

list_ds = tf.data.Dataset.list_files(data_dir_string + "/*/*")

val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
test_ds = list_ds.take(val_size)

def get_label(file_path):
  parts = tf.strings.split(file_path, '/')
  #print(file_path.numpy())
  return parts[-2] == CLASS_NAMES

def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=1) #grayscale images
  img = tf.image.convert_image_dtype(img, tf.float32) 
   #convert unit8 tensor to floats in the [0,1]range
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]) 
  #resize the image into 28*28

def process_path(file_path):
  label = get_label(file_path)
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE)

############labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
print(int(tf.data.experimental.cardinality(train_ds)))
print(int(tf.data.experimental.cardinality(test_ds)))

#print(list_ds)

#print(list(labeled_ds.as_numpy_iterator()))
#for el in labeled_ds.as_numpy_iterator():
    #print(el)
    
#print(tf.strings.as_string(labeled_ds))

#print(list(labeled_ds))

#for (image, label) in labeled_ds:
  #print("Image shape: ", image.numpy().shape)
  #print("Label: ", label.numpy())


from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

tf.keras.backend.clear_session()
print("TensorFlow Version: " + tf.__version__)

['without_mask' 'with_mask']
11042
8834
2208
TensorFlow Version: 2.1.0


## Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/)

[tf.keras.datasets.mnist](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data) returns: ***Tuple of Numpy arrays: (x_train, y_train), (x_test, y_test).***

## Defining the Model

In [3]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(8, 3, activation='relu')
    self.pool = MaxPooling2D()
    self.flatten = Flatten()
    self.d1 = Dense(16, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x0 = self.pool(x)
    x1 = self.conv1(x0)
    x2 = self.pool(x1)
    x3 = self.flatten(x2)
    x4 = self.d1(x3)
    return self.d2(x4)

model = MyModel()

## Loss Functions, and Optimisers

In [4]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [5]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Defining Training and Testing Steps

#### Training Step

In [6]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

#### Model Test Step

In [7]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

## Training Loop

In [8]:
EPOCHS = 15

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

InvalidArgumentError: slice index -1 of dimension 0 out of bounds.
	 [[{{node strided_slice}}]]

## Exporting to uTensor

A sample generator for the offline-quantizer to estimate the optimal quantization ranges for all activations.

In [ ]:
# representative data function
num_calibration_steps = 128
calibration_dtype = tf.float32

def representative_dataset_gen():
    for _ in range(num_calibration_steps):
        rand_idx = np.random.randint(0, x_test.shape[0]-1)
        sample = x_test[rand_idx]
        sample = sample[tf.newaxis, ...]
        sample = tf.cast(sample, dtype=calibration_dtype)
        yield [sample]

One-liner uTensor export API

In [ ]:
tflm_keras_export(
    model,
    representive_dataset=representative_dataset_gen,
    model_name='my_model',
    target='utensor',
)

In [ ]:
!pwd